##**Importing Libraries**


In [30]:
import pandas as pd
import numpy as np
from keras.utils import np_utils
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.regularizers import l2

###**Data Preprocessing**

In [31]:
!unzip 'fer2013'

Archive:  fer2013.zip
replace fer2013.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: fer2013.csv             


In [32]:
dataFrame = pd.read_csv('fer2013.csv')
print(dataFrame.shape)
y_train=dataFrame.iloc[:30000,:1].values
y_test =  dataFrame.iloc[30000:,:1].values
X=dataFrame.iloc[:30000,1:2].values
X_test = dataFrame.iloc[30000:,1:2].values
x_train,x_test = [],[]

for i in range(30000):
    if y_train[i][0]==1:
        y_train[i][0]=0
    if y_train[i][0]==4:
        y_train[i][0]=2
    if y_train[i][0]==5:
        y_train[i][0]=1
    if y_train[i][0]==6:
        y_train[i][0]=4
for i in range(5887):
    if y_test[i][0]==1:
        y_test[i][0]=0
    if y_test[i][0]==4:
        y_test[i][0]=2
    if y_test[i][0]==5:
        y_test[i][0]=1
    if y_test[i][0]==6:
        y_test[i][0]=4
        
y_train=np_utils.to_categorical(y_train)        
y_test=np_utils.to_categorical(y_test)        
for i in range(30000):
    pixels = np.array(X[i][0].split(" "),dtype='float32')
    pixels=np.reshape(pixels,(48,48,1))
    x_train.append(pixels)
for i in range(5887):
    pixels = np.array(X_test[i][0].split(" "),dtype='float32')
    pixels=np.reshape(pixels,(48,48,1))
    x_test.append(pixels)

x_train=np.array(x_train,'float32')/255.0
x_test=np.array(x_test,'float32')/255.0

(35887, 3)


## **Building the Model**


In [38]:
num_features=64


model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(num_features, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_56 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization_37 (Batc (None, 46, 46, 64)        256       
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_38 (Batc (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 20, 20, 128)     

### *Compling and Testing*

In [42]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])


hist=model.fit(x_train,y_train,shuffle=1,epochs=10,batch_size=128,validation_split=0.2,callbacks=[chkp])

Train on 24000 samples, validate on 6000 samples
Epoch 1/10
24000/24000 [==============================] - 38s 2ms/step - loss: 0.4379 - accuracy: 0.8342 - val_loss: 0.9393 - val_accuracy: 0.6855

Epoch 00001: val_accuracy did not improve from 0.70017
Epoch 2/10
24000/24000 [==============================] - 36s 1ms/step - loss: 0.4251 - accuracy: 0.8414 - val_loss: 0.8685 - val_accuracy: 0.6970

Epoch 00002: val_accuracy did not improve from 0.70017
Epoch 3/10
24000/24000 [==============================] - 36s 1ms/step - loss: 0.4259 - accuracy: 0.8386 - val_loss: 0.8619 - val_accuracy: 0.6992

Epoch 00003: val_accuracy did not improve from 0.70017
Epoch 4/10
24000/24000 [==============================] - 36s 1ms/step - loss: 0.4314 - accuracy: 0.8397 - val_loss: 0.8904 - val_accuracy: 0.6993

Epoch 00004: val_accuracy did not improve from 0.70017
Epoch 5/10
24000/24000 [==============================] - 36s 1ms/step - loss: 0.4085 - accuracy: 0.8478 - val_loss: 0.8891 - val_accuracy:

## *Saving the weights*

In [ ]:
model.load_weights('best_model.h5')

## Callbacks

In [35]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
es = EarlyStopping(monitor='val_acc',patience=2)
chkp = ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

###*Evaluating the model*


In [46]:
loss,acc = model.evaluate(x_test,y_test)
acc

5887/5887 [==============================] - 3s 580us/step


0.6835399866104126